In [1]:

import numpy as np # for data manipulation
print('numpy: %s' % np.__version__) # print version
import math # to help with data reshaping of the data

import numpy as np
import torch
#import torchvision
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

# from sklearn.model_selection import train_test_split
import tqdm
import matplotlib.pyplot as plt
import logging

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import os
os.chdir('../')

from pose_gru import PoseGRU_inputFC2
from benji_prox_dataloader import *

numpy: 1.21.5


In [6]:
root_dir = "D:/prox_data/joint_locations/"
# root_dir = "/cluster/scratch/bdayan/prox_data/joint_locations/"

batch_size = 15

pd = proxDatasetSkeleton(root_dir=root_dir, output_type='joint_locations')
dataloader = DataLoader(pd, batch_size=batch_size,
                        shuffle=True, num_workers=0, collate_fn=my_collate)

TypeError: stat: path should be string, bytes, os.PathLike or integer, not NoneType

In [14]:
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(in_data[0].detach().numpy())
o3d.visualization.draw_geometries([pcd])

In [5]:
help(proxDatasetSkeleton)

Help on class proxDatasetSkeleton in module benji_prox_dataloader:

class proxDatasetSkeleton(DatasetBase)
 |  proxDatasetSkeleton(output_type='joint_locations', smplx_model_path=None, **kwargs)
 |  
 |  Method resolution order:
 |      proxDatasetSkeleton
 |      DatasetBase
 |      torch.utils.data.dataset.Dataset
 |      typing.Generic
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __getitem__(self, idx)
 |  
 |  __init__(self, output_type='joint_locations', smplx_model_path=None, **kwargs)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  ----------------------------------------------------------------------
 |  Data and other attributes defined here:
 |  
 |  __parameters__ = ()
 |  
 |  ----------------------------------------------------------------------
 |  Methods inherited from DatasetBase:
 |  
 |  __len__(self)
 |  
 |  ----------------------------------------------------------------------
 |  Methods inherited from torch.util

In [101]:
root_dir = 'D:/prox_data/PROXD_attempt2/PROXD'
smplx_model_path='../models_smplx_v1_1/models/'
in_frames = 10
pred_frames = 5
batch_size = 15
pd = proxDataset(root_dir, in_frames=in_frames, pred_frames=pred_frames, output_type='raw_pkls', smplx_model_path=smplx_model_path)


dataloader = DataLoader(pd, batch_size=1,
                        shuffle=True, num_workers=0, collate_fn=my_collate)

In [4]:
i, in_data, pred_data = pd.__getitem__(5)

In [6]:
in_data.shape

torch.Size([10, 25, 3])

In [7]:
data = in_data[0]

In [8]:
import open3d as o3d
import smplx

In [9]:
import numpy as np
import open3d as o3d
import torch


# test whether GPU works
try:
  aa = torch.cuda.FloatTensor(3,3).normal_()
except:
  # raise OSError('-- cuda is not supported')
  pass

# specify device
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')


print('-- pytorch version={}'.format(torch.__version__))
print('-- device={}'.format(device))

smplx_model_path='../models_smplx_v1_1/models/'

import smplx  # SMPL-X; pip install smplx
from human_body_prior.tools.model_loader import load_model


def get_smplx_model(gender, device):
    body_model_path = smplx_model_path
    body_model = smplx.create(body_model_path, 
                          model_type='smplx',        ## smpl, smpl+h, or smplx?
                          gender=gender, ext='npz',  ## file format 
                          num_pca_comps=12,          ## MANO hand pose pca component
                          create_global_orient=True, 
                          create_body_pose=True,
                          create_betas=True,
                          create_left_hand_pose=True,
                          create_right_hand_pose=True,
                          create_expression=True, 
                          create_jaw_pose=True,
                          create_leye_pose=True,
                          create_reye_pose=True,
                          create_transl=True,
                          batch_size=1               ## how many bodies in a batch?
                          )
    body_model.eval()
    return body_model

smplx_neutral = get_smplx_model(gender='neutral', device=device)


def draw_bodies(gender, betas, thetas):
    bm = smplx_neutral


    ## read verts and face from smplx model
    params = {'betas': betas,
              'body_pose': thetas
              }


    verts = bm(return_verts=True, **params).vertices.detach().cpu().numpy()[0]
    faces = bm.faces

    ## put verts and face into open3d, and compute surface normal
    coord = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.5)
    body = o3d.geometry.TriangleMesh()
    body.vertices = o3d.utility.Vector3dVector(verts)
    body.triangles = o3d.utility.Vector3iVector(faces)
    body.vertex_normals = o3d.utility.Vector3dVector([])
    body.triangle_normals = o3d.utility.Vector3dVector([])
    body.compute_vertex_normals()
    o3d.visualization.draw_geometries([body, coord])
    
    return None


-- pytorch version=1.10.0
-- device=cpu


In [11]:
data

{'camera_rotation': array([[[1., 0., 0.],
         [0., 1., 0.],
         [0., 0., 1.]]], dtype=float32),
 'camera_translation': array([[0., 0., 0.]], dtype=float32),
 'betas': array([[ 0.22432968, -0.64445114,  1.2173733 ,  0.11142754,  0.280056  ,
         -0.740762  ,  0.1824827 ,  0.728877  , -0.01722069, -1.3591751 ]],
       dtype=float32),
 'global_orient': array([[ 1.1642715, -0.5064939,  2.978201 ]], dtype=float32),
 'transl': array([[-0.29386428,  0.2872536 ,  2.1008468 ]], dtype=float32),
 'left_hand_pose': array([[ 0.05242801, -0.25586367, -0.12669666,  0.02269641, -0.10736813,
         -0.03054625, -0.02508651, -0.09175161,  0.01792343, -0.09695002,
          0.04674345,  0.03218148]], dtype=float32),
 'right_hand_pose': array([[-0.7191012 ,  0.08357322, -0.2126172 , -0.14656302,  0.26723054,
          0.15955654,  0.12229528, -0.11545242, -0.15355091, -0.19015649,
          0.7293587 , -0.10225791]], dtype=float32),
 'jaw_pose': array([[-4.8616584e-04, -1.2542189e-05, -3.

In [80]:
outs = []
for data in in_data[1]:
    betas = torch.Tensor(data['betas'])
    body_pose = torch.Tensor(data['body_pose'])
    global_orient= torch.Tensor(data['global_orient'])
    transl=torch.Tensor(data['transl'])
    out = smplx_neutral(return_joints=True, betas=betas, body_pose=body_pose, global_orient=global_orient, transl=transl)
    joints = out.joints[:, :25]
    outs.append(joints)

In [91]:
in_data[0]
foo = Path(in_data[0][0])
foo.parent
pd.root_dir

['D:\\prox_data\\PROXD_attempt2\\PROXD\\BasementSittingBooth_00142_01\\results\\s001_frame_00076__00.00.02.499\\000.pkl',
 'D:\\prox_data\\PROXD_attempt2\\PROXD\\BasementSittingBooth_00142_01\\results\\s001_frame_00077__00.00.02.532\\000.pkl',
 'D:\\prox_data\\PROXD_attempt2\\PROXD\\BasementSittingBooth_00142_01\\results\\s001_frame_00078__00.00.02.563\\000.pkl',
 'D:\\prox_data\\PROXD_attempt2\\PROXD\\BasementSittingBooth_00142_01\\results\\s001_frame_00079__00.00.02.599\\000.pkl',
 'D:\\prox_data\\PROXD_attempt2\\PROXD\\BasementSittingBooth_00142_01\\results\\s001_frame_00080__00.00.02.632\\000.pkl',
 'D:\\prox_data\\PROXD_attempt2\\PROXD\\BasementSittingBooth_00142_01\\results\\s001_frame_00081__00.00.02.666\\000.pkl',
 'D:\\prox_data\\PROXD_attempt2\\PROXD\\BasementSittingBooth_00142_01\\results\\s001_frame_00082__00.00.02.700\\000.pkl',
 'D:\\prox_data\\PROXD_attempt2\\PROXD\\BasementSittingBooth_00142_01\\results\\s001_frame_00083__00.00.02.733\\000.pkl',
 'D:\\prox_data\\PROXD_a

WindowsPath('D:/prox_data/PROXD_attempt2/PROXD/BasementSittingBooth_00142_01/results/s001_frame_00076__00.00.02.499')

WindowsPath('D:/prox_data/PROXD_attempt2/PROXD')

In [85]:
pd.root_dir

WindowsPath('D:/prox_data/PROXD_attempt2/PROXD')

In [93]:
foo.relative_to(pd.root_dir)

WindowsPath('BasementSittingBooth_00142_01/results/s001_frame_00076__00.00.02.499/000.pkl')

In [96]:
pd.root_dir / '../../'

WindowsPath('D:/prox_data/PROXD_attempt2/PROXD/../..')

In [99]:
pd.root_dir
foo
foo.parent

WindowsPath('D:/prox_data/PROXD_attempt2/PROXD')

WindowsPath('D:/prox_data/PROXD_attempt2/PROXD/BasementSittingBooth_00142_01/results/s001_frame_00076__00.00.02.499/000.pkl')

WindowsPath('D:/prox_data/PROXD_attempt2/PROXD/BasementSittingBooth_00142_01/results/s001_frame_00076__00.00.02.499')

In [86]:
pd.sequences[0]

['BasementSittingBooth_00142_01',
 [{'fn': 'D:\\prox_data\\PROXD_attempt2\\PROXD\\BasementSittingBooth_00142_01\\results\\s001_frame_00001__00.00.00.029\\000.pkl',
   'frame': 1,
   'tstamp': datetime.time(0, 0, 0, 29000)},
  {'fn': 'D:\\prox_data\\PROXD_attempt2\\PROXD\\BasementSittingBooth_00142_01\\results\\s001_frame_00002__00.00.00.050\\000.pkl',
   'frame': 2,
   'tstamp': datetime.time(0, 0, 0, 50000)},
  {'fn': 'D:\\prox_data\\PROXD_attempt2\\PROXD\\BasementSittingBooth_00142_01\\results\\s001_frame_00003__00.00.00.074\\000.pkl',
   'frame': 3,
   'tstamp': datetime.time(0, 0, 0, 74000)},
  {'fn': 'D:\\prox_data\\PROXD_attempt2\\PROXD\\BasementSittingBooth_00142_01\\results\\s001_frame_00004__00.00.00.107\\000.pkl',
   'frame': 4,
   'tstamp': datetime.time(0, 0, 0, 107000)},
  {'fn': 'D:\\prox_data\\PROXD_attempt2\\PROXD\\BasementSittingBooth_00142_01\\results\\s001_frame_00005__00.00.00.140\\000.pkl',
   'frame': 5,
   'tstamp': datetime.time(0, 0, 0, 140000)},
  {'fn': 'D:\\

In [82]:
outs[0].shape

torch.Size([1, 25, 3])

In [53]:
g1 = in_data[1][0]['global_orient']
g2 = in_data[1][1]['global_orient']

In [58]:
g1
g2
from pytorch3d import transforms

array([[ 1.1642715, -0.5064939,  2.978201 ]], dtype=float32)

array([[ 1.242714 , -0.5080164,  2.951133 ]], dtype=float32)

In [67]:

# r1 = o3d.geometry.Geometry3D.get_rotation_matrix_from_xyz(g1.reshape(3, 1))
# r2 = o3d.geometry.Geometry3D.get_rotation_matrix_from_xyz(g2.reshape(3, 1))

r1 = transforms.euler_angles_to_matrix(torch.Tensor(g1), convention='XYZ')
r2 = transforms.euler_angles_to_matrix(torch.Tensor(g2), convention='XYZ')


In [72]:
g1

array([[ 1.1642715, -0.5064939,  2.978201 ]], dtype=float32)

In [69]:
r1.shape
r1.transpose(1, 2) @ r1

torch.Size([1, 3, 3])

tensor([[[1.0000e+00, 0.0000e+00, 2.8871e-08],
         [0.0000e+00, 1.0000e+00, 0.0000e+00],
         [2.8871e-08, 0.0000e+00, 1.0000e+00]]])

In [71]:
transforms.matrix_to_euler_angles(r2 @ r1.transpose(1, 2), convention='XYZ')

tensor([[ 0.0917,  0.0211, -0.0109]])

In [73]:
transforms.matrix_to_euler_angles(r1 @ r1.transpose(1, 2), convention='XYZ')

tensor([[-0.0000e+00, -1.4901e-08, -2.9802e-08]])

In [39]:
joined = np.concatenate([outs[0], outs[1]], axis=0)

In [11]:
in_data

tensor([[[-1.4309e-01, -3.5130e-01,  1.0567e-01],
         [-6.9707e-02, -4.0782e-01,  4.3905e-02],
         [-1.8040e-01, -4.5087e-01,  4.7406e-02],
         [-1.7488e-01, -2.4934e-01,  1.2384e-01],
         [ 5.2975e-02, -6.9828e-01, -1.7176e-01],
         [-1.2623e-01, -8.1301e-01,  5.9472e-02],
         [-1.9988e-01, -1.3985e-01,  1.9135e-01],
         [ 9.7733e-02, -9.7865e-01, -4.5315e-01],
         [-3.4260e-02, -1.1827e+00, -7.6900e-02],
         [-2.1865e-01, -1.1412e-01,  2.3666e-01],
         [ 1.6502e-01, -1.0468e+00, -3.5868e-01],
         [-6.1624e-02, -1.2574e+00,  2.9704e-02],
         [-2.6738e-01,  3.1739e-02,  3.0126e-01],
         [-1.9631e-01, -3.0753e-02,  2.7425e-01],
         [-2.7859e-01, -5.0379e-02,  2.7117e-01],
         [-2.4078e-01,  1.3073e-01,  4.3743e-01],
         [-1.0207e-01,  4.1182e-02,  2.9747e-01],
         [-3.7692e-01, -3.2577e-02,  3.0497e-01],
         [-1.4706e-02, -1.4403e-01,  1.3179e-01],
         [-4.3719e-01, -2.5505e-01,  1.6446e-01],


In [13]:
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(in_data[0].detach().numpy())
o3d.visualization.draw_geometries([pcd])

In [2]:
joints

NameError: name 'joints' is not defined

In [ ]:
gender = 'neutral'
betas = torch.FloatTensor(1,10).zero_()
# thetas = vposer.decode(torch.FloatTensor(1,32).normal_(), output_type='aa')
thetas = torch.FloatTensor(1, 21, 3).zero_()

draw_bodies(gender, betas, thetas)

In [116]:
p = Path(r'D:\prox_data\joint_locations\BasementSittingBooth_00142_01\results\s001_frame_00010__00.00.00.305\000.pkl')
with open(r'D:\prox_data\joint_locations\BasementSittingBooth_00142_01\results\s001_frame_00010__00.00.00.305\000.pkl', 'rb') as file:
    temp = pickle.load(file)

In [117]:
temp

{'in_fns': ['D:\\prox_data\\PROXD_attempt2\\PROXD\\BasementSittingBooth_00142_01\\results\\s001_frame_00001__00.00.00.029\\000.pkl',
  'D:\\prox_data\\PROXD_attempt2\\PROXD\\BasementSittingBooth_00142_01\\results\\s001_frame_00002__00.00.00.050\\000.pkl',
  'D:\\prox_data\\PROXD_attempt2\\PROXD\\BasementSittingBooth_00142_01\\results\\s001_frame_00003__00.00.00.074\\000.pkl',
  'D:\\prox_data\\PROXD_attempt2\\PROXD\\BasementSittingBooth_00142_01\\results\\s001_frame_00004__00.00.00.107\\000.pkl',
  'D:\\prox_data\\PROXD_attempt2\\PROXD\\BasementSittingBooth_00142_01\\results\\s001_frame_00005__00.00.00.140\\000.pkl',
  'D:\\prox_data\\PROXD_attempt2\\PROXD\\BasementSittingBooth_00142_01\\results\\s001_frame_00006__00.00.00.192\\000.pkl',
  'D:\\prox_data\\PROXD_attempt2\\PROXD\\BasementSittingBooth_00142_01\\results\\s001_frame_00007__00.00.00.212\\000.pkl',
  'D:\\prox_data\\PROXD_attempt2\\PROXD\\BasementSittingBooth_00142_01\\results\\s001_frame_00008__00.00.00.241\\000.pkl',
  'D:\

In [126]:
a, b = temp['in_joint_locations'], temp['pred_joint_locations']
a[0]
a.shape
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(np.concatenate([a[-2].detach().numpy(), a[-1].detach().numpy()], axis=0))
o3d.visualization.draw_geometries([pcd])

tensor([[-0.0793, -0.3643, -0.0065],
        [ 0.0019, -0.4264, -0.0635],
        [-0.1156, -0.4719, -0.0657],
        [-0.1137, -0.2532,  0.0073],
        [ 0.1609, -0.7483, -0.1987],
        [-0.0530, -0.7553, -0.2893],
        [-0.1397, -0.1278,  0.0670],
        [ 0.1915, -1.1004, -0.3962],
        [ 0.0693, -1.0991, -0.4749],
        [-0.1601, -0.0900,  0.1138],
        [ 0.2967, -1.1270, -0.3082],
        [-0.0264, -1.1608, -0.3975],
        [-0.2185,  0.0698,  0.1510],
        [-0.1376,  0.0035,  0.1413],
        [-0.2303, -0.0212,  0.1363],
        [-0.1946,  0.2015,  0.2573],
        [-0.0353,  0.0929,  0.1427],
        [-0.3464, -0.0016,  0.1509],
        [ 0.0479, -0.1223,  0.0087],
        [-0.3563, -0.2643,  0.0736],
        [ 0.1711, -0.3284, -0.0585],
        [-0.3611, -0.5015,  0.0078],
        [-0.2103,  0.1975,  0.2599],
        [-0.1480,  0.2287,  0.3171],
        [-0.1990,  0.2080,  0.3455]], grad_fn=<SelectBackward0>)

torch.Size([10, 25, 3])

In [108]:
from pathlib import PurePath
PurePath(p)

PureWindowsPath('D:/prox_data/PROXD_attempt2/PROXD/../../joint_locations/BasementSittingBooth_00142_01\results/s001_frame_00010__00.00.00.305\x00.pkl')

In [110]:
'asdf'.replace('s', 'a')

'aadf'

In [140]:
a, b, c = pd.__getitem__(2098)
b[1]

[{'camera_rotation': array([[[1., 0., 0.],
          [0., 1., 0.],
          [0., 0., 1.]]], dtype=float32),
  'camera_translation': array([[0., 0., 0.]], dtype=float32),
  'betas': array([[-0.6387518 , -0.32014745,  0.5598045 , -0.2579955 ,  0.08565714,
          -0.20114422,  0.47912633,  0.07306933,  0.20813356,  0.0459968 ]],
        dtype=float32),
  'global_orient': array([[-0.09109784, -0.55160886,  3.0707147 ]], dtype=float32),
  'transl': array([[-2.1276882 ,  0.00446024,  3.2431786 ]], dtype=float32),
  'left_hand_pose': array([[-0.0128428 , -0.01061214, -0.00751931, -0.00757575,  0.00211687,
          -0.0001624 ,  0.00360943, -0.00242291,  0.00068849, -0.00094684,
           0.00318422, -0.00100048]], dtype=float32),
  'right_hand_pose': array([[-5.8118412e-03,  4.7094817e-03, -2.2435416e-03,  7.1145534e-03,
           2.0538245e-03,  5.6759763e-04,  1.5468204e-03, -5.7423062e-04,
          -4.6068268e-05, -3.3205599e-04,  1.0584807e-03,  6.8104011e-04]],
        dtype=floa

In [142]:
b[1][-1]

In [134]:
b[1]

[{'camera_rotation': array([[[1., 0., 0.],
          [0., 1., 0.],
          [0., 0., 1.]]], dtype=float32),
  'camera_translation': array([[0., 0., 0.]], dtype=float32),
  'betas': array([[ 0.53186613, -1.2437785 ,  2.0492089 ,  0.04952427,  0.15782367,
          -0.27313647,  0.49754414,  0.19092573,  0.14918391, -0.11288952]],
        dtype=float32),
  'global_orient': array([[-2.0463557 ,  0.43267703, -1.975896  ]], dtype=float32),
  'transl': array([[-0.58156836,  0.0500098 ,  2.9849393 ]], dtype=float32),
  'left_hand_pose': array([[ 0.0012432 , -0.01782784,  0.0037376 ,  0.00226955,  0.01040434,
           0.00495332, -0.00287453, -0.00962047, -0.00221348, -0.0047482 ,
           0.01231256, -0.00166697]], dtype=float32),
  'right_hand_pose': array([[-1.3423636 , -0.5549602 ,  0.4053884 , -0.6626304 ,  0.26504517,
          -0.08826961,  0.01689136,  0.28352997,  0.01431077,  0.15724449,
          -0.2273408 ,  0.01636385]], dtype=float32),
  'jaw_pose': array([[-0.00125205,  0.